In [22]:
import pandas as pd
from timeSeriesToSupervised import series_to_supervised
import numpy as np
import splitSequence as sq
import h2o

In [9]:
symbol = 'cah'
stock = pd.read_csv(f'../inputData/stocksMid/{symbol}.us.txt', parse_dates=True).drop(columns='OpenInt')
stock.set_index('Date', inplace=True)
display(stock.head())
print(stock.shape)

,Open,High,Low,Close,Volume
Date,,,,,
1988-01-04,0.6326,0.6326,0.6326,0.6326,40783
1988-01-05,0.6500,0.6500,0.6500,0.6500,177705
1988-01-06,0.6500,0.6758,0.6326,0.6758,110703
1988-01-07,0.6758,0.6931,0.6584,0.6931,91766
1988-01-08,0.7105,0.7366,0.7105,0.7105,152938


(7519, 5)


In [35]:
df_forML = series_to_supervised(list(cah['Close']), n_in=50, n_out=1, name_var='Close')
df_forML.rename(columns={'Close(t)':'y'}, inplace=True) 
for col in ['Open', 'High', 'Low', 'Volume']:
    df_forML = df_forML.merge(series_to_supervised(list(cah[col]), n_in=50, n_out=0, name_var=col), left_index=True, right_index=True)
df_forML = df_forML.set_index(stock.iloc[50:].index)
df_forML = df_forML.iloc[:int(len(df_forML) * 0.8)]
df_forML.head()

,Close(t-50),Close(t-49),Close(t-48),Close(t-47),Close(t-46),Close(t-45),Close(t-44),Close(t-43),Close(t-42),Close(t-41),...,Volume(t-10),Volume(t-9),Volume(t-8),Volume(t-7),Volume(t-6),Volume(t-5),Volume(t-4),Volume(t-3),Volume(t-2),Volume(t-1)
Date,,,,,,,,,,,,,,,,,,,,,
1988-03-15,0.6326,0.6500,0.6758,0.6931,0.7105,0.7539,0.7539,0.7366,0.7366,0.7539,...,512721.0,4369.0,907461.0,27676.0,17477.0,184986.0,260728.0,10199.0,21847.0,1609540.0
1988-03-16,0.6500,0.6758,0.6931,0.7105,0.7539,0.7539,0.7366,0.7366,0.7539,0.7366,...,4369.0,907461.0,27676.0,17477.0,184986.0,260728.0,10199.0,21847.0,1609540.0,270926.0
1988-03-17,0.6758,0.6931,0.7105,0.7539,0.7539,0.7366,0.7366,0.7539,0.7366,0.7366,...,907461.0,27676.0,17477.0,184986.0,260728.0,10199.0,21847.0,1609540.0,270926.0,110703.0
1988-03-18,0.6931,0.7105,0.7539,0.7539,0.7366,0.7366,0.7539,0.7366,0.7366,0.7105,...,27676.0,17477.0,184986.0,260728.0,10199.0,21847.0,1609540.0,270926.0,110703.0,266556.0
1988-03-21,0.7105,0.7539,0.7539,0.7366,0.7366,0.7539,0.7366,0.7366,0.7105,0.7105,...,17477.0,184986.0,260728.0,10199.0,21847.0,1609540.0,270926.0,110703.0,266556.0,521463.0


In [30]:
dfh2o = h2o.H2OFrame(df_forML)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [32]:
train, test = dfh2o.split_frame(ratios=[0.8])

In [25]:
h2o.init(   nthreads=7,
            max_mem_size='5G')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu119.10.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu119.10.1, mixed mode, sharing)
  Starting server from /home/edu/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp90fhtrqq
  JVM stdout: /tmp/tmp90fhtrqq/h2o_edu_started_from_python.out
  JVM stderr: /tmp/tmp90fhtrqq/h2o_edu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,1 month and 4 days
H2O cluster name:,H2O_from_python_edu_knukjn
H2O cluster total nodes:,1
H2O cluster free memory:,5 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,7
H2O cluster status:,"accepting new members, healthy"


In [ ]:
h2o.automl()

In [ ]:
h2o.shutdown()